In [108]:

import os
from itertools import combinations
from get_input import get_input

os.chdir('/Users/soup/Documents/projects/advent-of-code')

In [111]:
rings = ['Dmg+1', 'Dmg+2', 'Dmg+3', 'Def+1', 'Def+2', 'Def+3', '', '']

for duo in combinations(rings, 2):
    print(type(duo))

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
